# Classificação, Casting e Categorias

Neste caderno, abordaremos operações de ordenação, bem como dados categóricos. Também aprenderemos como usar funções apply para realizar operações mais personalizadas no Pandas.

Vamos importar `pandas` e, em seguida, declarar este pequeno dataframe weathers para praticar.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "id_registro": ['DCMXP87EDE', 'ZMIFM3HX9G', 'HIVVXBAPS2', 'U1AA66UDES', 'B20KL5PW3L', 'FIZLY34KSQ'],
    "plegadas_chuva": [1.1, 0.0, 0.0, 2.4, 11.2, 3.2],
    "tornado": [0, 1, 0, 0, 0, 0],
    "raios": [0, 1, 1, 1, 0, 0],
    "velocidade_vento_mph": [3.1, 143.0, 12.2, 8.1, 5.0, 19.0],
    "gravidade": ['LIMPO', 'SEVERO', 'LEVE', 'LEVE', 'GRAVE', 'LIMPO']
})

df

## Tipos de Dados e Casting

À medida que falamos mais sobre limpeza de dados, talvez seja um bom momento para falar sobre tipos de dados e sua escolha criteriosa. Alguns tipos de dados básicos do Pandas, com exemplos, são declarados abaixo. Esses são os tipos de dados que também fazem parte do NumPy e, portanto, são os mais comuns.

In [ ]:
df_tipos = pd.DataFrame({
    'flutuante': [1.2],
    'inteiro': [3],
    'data_hora': [pd.Timestamp('20230130')],
    'booleano': [True],
    'diferenca_tempo': [pd.Timestamp('20230130') - pd.Timestamp('20230127')],
    'texto': ['olá']
})

df_tipos


Você pode visualizar os tipos de dados de um determinado dataframe usando a propriedade `dtypes`.

In [ ]:
df_tipos.dtypes

Agora vamos voltar nossa atenção para o pequeno conjunto de dados meteorológicos.

In [ ]:
df

Vamos observar os tipos de dados.

In [ ]:
df.dtypes

Uma das operações mais básicas na limpeza de dados é a conversão. Você pode usar a função `astype()` para converter uma determinada coluna para outro tipo de dado. Por exemplo, as colunas `tornado` e `raios` têm apenas 1 e 0, indicando que devem ser valores booleanos (True=1, False=0). Podemos convertê-los para booleanos aqui.

In [ ]:
df['tornado'] = df['tornado'].astype('bool')
df['raios'] = df['raios'].astype('bool')

df


E com certeza, você verá que os tipos de dados foram alterados para `book` para essas duas colunas.

In [ ]:
df.dtypes

## Classificando Valores

No Pandas, você pode classificar os dados ao longo de uma linha ou coluna especificando seu eixo na função `sort_values()`. Abaixo, classificamos em ordem crescente primeiro pelo campo `raios`, seguido pelo campo `polegadas_chuva`.

In [ ]:
df.sort_values(by=["raios", "polegadas_chuva"])

Se eu quiser comportamentos de classificação diferentes para cada coluna, com algumas em ordem crescente e outras em ordem decrescente, passe uma lista booleana para o parâmetro `ascending`. Abaixo, definimos `raios` como decrescente, de modo que os registros `True` subam para o topo, enquanto `polegadas_chuva` é crescente.

In [ ]:
df.sort_values(by=["raios","polegadas_chuva"],ascending=[False,True])

> Ao usar os métodos de classificação, lembre-se de adicionar o parâmetro `inplace=True` se quiser substituir o dataframe existente pelo classificado.

## Índice de classificação

Vamos demonstrar como classificar em um índice. Vamos primeiro definir o índice para usar o `record_id` para as linhas

In [ ]:
df.set_index('record_id', inplace=True)

df

Agora, quando classificamos as linhas (usando `axis=0`), observe que agora classificamos em ordem alfabética pelo `record_id` como índice.

In [ ]:
df.sort_index(axis=0)

Isso pode não parecer muito interessante, pois também poderíamos ter ordenado `record_id` como uma coluna. Mas agora considere que, se definirmos `axis=1` em `sort_index()`, podemos ordenar as colunas!

In [ ]:
df.sort_index(axis=1)

Isso pode ser útil para classificar as colunas, e você pode classificar apenas determinadas colunas extraindo um dataframe parcial e, em seguida, substituindo essas colunas. Observe que, como o `record_id` foi transformado em um índice para as linhas, ele não é classificado com o restante das colunas e permanece à esquerda do dataframe.

> Ao usar os métodos de classificação, lembre-se de adicionar o parâmetro `inplace=True` se desejar substituir o dataframe existente pelo classificado.

## Categorias

Às vezes, haverá colunas em um dataframe que permitem apenas alguns valores. Quando esses valores são strings, torna-se ainda mais importante considerar convertê-los em um tipo de categoria. Nos bastidores, isso melhorará o desempenho do dataframe e eliminará redundância devido a strings duplicadas.

Em nosso conjunto de dados meteorológicos, observe a coluna `gravidade`. Digamos que os únicos valores possíveis para ela sejam "LIMPO", "LEVE", "GRAVE" e "SEVERO". Em vez de armazená-los como strings, podemos transformá-los explicitamente em categorias.

Primeiro, podemos criar um novo `CategoricalDType` e especificar as `categories` esperadas em uma lista. Se quisermos que as categorias tenham uma noção de ordenação, podemos especificar `ordered=True` e esses rótulos nessa ordem se tornarão a hierarquia. Em ordem crescente, "LIMPO" vem antes de "LEVE", depois "LIMPO" vem antes de "GRAVE" e assim por diante...

In [ ]:
tipo_categoria = pd.CategoricalDtype(categories=["LIMPO", "LEVE", "GRAVE", "SEVERO"], ordered=True)

Podemos então passar essa instância de `CategoricalDType` para a função `astype()` em um dataframe e substituir essa coluna pela `severity` categorizada.

In [ ]:
df["gravidade"] = df["gravidade"].astype(tipo_categoria)

df

Com certeza, se você inspecionar os tipos de dados do dataframe, a coluna `gravidade` agora é do tipo `category`. Isso será muito mais eficiente para trabalhar.

In [ ]:
df.dtypes

Observe que, se você aplicar uma categorização a uma coluna que possui valores que não correspondem a nenhuma categoria, esses valores se tornarão valores `NA`.

Por fim, observe que, ao classificar na coluna `gravidade`, ela não será mais classificada em ordem alfabética, mas sim na ordem de classificação definida em `CategoricalDtype`. Isso é comprovado pelo uso de `LEVE` antes de `GRAVE`.

In [ ]:
df.sort_values(by=["gravidade"])

# Usando apply()

Digamos que você queira categorizar as velocidades do vento, então crie esta função Python.

In [ ]:
def classificar_velocidade_vento(x):
    if x >= 60:
        return 'PERIGOSO'
    elif x >= 30:
        return 'ALTO'
    elif x >= 15:
        return 'MODERADO'
    else:
        return 'BAIXO'


Como aplicar isso à coluna `velocidade_vento_mph` e criar uma nova coluna a partir dela? Você pode usar a função `apply()`

In [ ]:
df['velocidade_vento_mph'].apply(classificar_velocidade_vento)

Você pode então anexar isso como uma nova coluna como uma categoria wind_speed_category.

In [ ]:
df["categoria_velocidade_vento"] = df['velocidade_vento_mph'].apply(classificar_velocidade_vento)

df

Use `apply()` para passar uma coluna de valores por meio de uma função, passe cada valor respectivo por ela e obtenha a saída.

## Exercício

Pegue o dataframe meteorológico que acabamos de criar e transforme `categoria_velocidade_vento` (que atualmente é armazenado como objetos string) em um tipo de categoria. Defina-o de forma que a ordem seja crescente: `BAIXO`, `MODERADO`, `ALTO` e, por fim, `PERIGOSO`. Em seguida, classifique essa coluna em ordem decrescente.

In [ ]:
tipo_categoria_vento = ?

df["categoria_velocidade_vento"] = df['categoria_velocidade_vento'].astype(?)

df.sort_values(by=?, ascending=?, inplace=True)
df


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
tipo_categoria_vento = pd.CategoricalDtype(categories=["BAIXO", "MODERADO", "ALTO", "PERIGOSO"], ordered=True)

df["categoria_velocidade_vento"] = df["categoria_velocidade_vento"].astype(tipo_categoria_vento)

df.sort_values(by=["categoria_velocidade_vento"], ascending=[False], inplace=True)

df